In [35]:
import os
import sys
sys.path.insert(0, '../')
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection
from colbert import Indexer, Searcher

In [36]:
fld_pth = "/home/sondors/Documents/price/BERT_data/data/metrics_data"
models = f'{fld_pth}/models.tsv'
offers = f'{fld_pth}/offers.tsv'

offers = Queries(path=offers)
models = Collection(path=models)

f'Loaded {len(offers)} queries and {len(models):,} passages'

[Dec 21, 18:04:33] #> Loading the queries from /home/sondors/Documents/price/BERT_data/data/metrics_data/offers.tsv ...
[Dec 21, 18:04:34] #> Got 135896 queries. All QIDs are unique.

[Dec 21, 18:04:34] #> Loading collection...
0M 


'Loaded 135896 queries and 103,214 passages'

## Indexing

For efficient search, we can pre-compute the ColBERT representation of each passage and index them.

Below, the `Indexer` take a model checkpoint and writes a (compressed) index to disk. We then prepare a `Searcher` for retrieval from this index.

(With four Titan V GPUs, indexing should take about 13 minutes. The output is fairly long/ugly at the moment!)

In [37]:
nbits = 2   # encode each dimension with 2 bits
doc_maxlen = 180   # truncate passages at 300 tokens

checkpoint = '/home/sondors/colbert'
index_name = f'models.18_categories.{nbits}bits'

with Run().context(RunConfig(nranks=1, experiment='notebook', gpus=0)):  # nranks specifies the number of GPUs to use.
    config = ColBERTConfig(doc_maxlen=doc_maxlen, nbits=nbits)

    indexer = Indexer(checkpoint=checkpoint, config=config)
    indexer.index(name=index_name, collection=models, overwrite=True)
indexer.get_index() # You can get the absolute path of the index, if needed.



[Dec 21, 18:04:34] #> Note: Output directory /home/sondors/Documents/price/ColBERT/docs/experiments/notebook/indexes/models.18_categories.2bits already exists


#> Starting...


 98%|█████████▊| 49/50 [02:19<00:03,  3.10s/it]

{
    "query_token_id": "[unused0]",
    "doc_token_id": "[unused1]",
    "query_token": "[Q]",
    "doc_token": "[D]",
    "ncells": null,
    "centroid_score_threshold": null,
    "ndocs": null,
    "load_index_with_mmap": false,
    "index_path": null,
    "nbits": 2,
    "kmeans_niters": 4,
    "resume": false,
    "similarity": "cosine",
    "bsize": 64,
    "accumsteps": 1,
    "lr": 1e-5,
    "maxsteps": 500000,
    "save_every": 1,
    "warmup": null,
    "warmup_bert": null,
    "relu": false,
    "nway": 2,
    "use_ib_negatives": false,
    "reranker": false,
    "distillation_alpha": 1.0,
    "ignore_scores": false,
    "model_name": null,
    "query_maxlen": 32,
    "attend_to_mask_tokens": false,
    "interaction": "colbert",
    "dim": 128,
    "doc_maxlen": 180,
    "mask_punctuation": true,
    "checkpoint": "\/home\/sondors\/colbert",
    "triples": "\/mnt\/vdb1\/Datasets\/18_categories\/ColBERT_dataset\/triples.json",
    "collection": "\/home\/sondors\/Documents\/pr

100%|██████████| 50/50 [02:22<00:00,  2.86s/it]
/home/sondors/anaconda3/envs/torch_gpu/lib/python3.8/site-packages/torch/cuda/amp/grad_scaler.py:118: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


[Dec 21, 18:04:43] [0] 		 # of sampled PIDs = 56310 	 sampled_pids[:3] = [54607, 96034, 1332]
[Dec 21, 18:04:43] [0] 		 #> Encoding 56310 passages..


  0%|          | 0/50 [00:00<?, ?it/s]/home/sondors/anaconda3/envs/torch_gpu/lib/python3.8/site-packages/torch/amp/autocast_mode.py:202: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
 14%|█▍        | 7/50 [00:43<04:33,  6.35s/it]]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:11                                                                                   │
│                                                                                                  │
│    8 │   config = ColBERTConfig(doc_maxlen=doc_maxlen, nbits=nbits)                              │
│    9 │                                                                                           │
│   10 │   indexer = Indexer(checkpoint=checkpoint, config=config)                                 │
│ ❱ 11 │   indexer.index(name=index_name, collection=models, overwrite=True)                       │
│   12 indexer.get_index() # You can get the absolute path of the index, if needed.                │
│   13                                                                                             │
│                                                                                                  │
│ /home/sondors/Documents/price/ColBERT/docs/../colbert/indexer.py:74 in index                     │
│                                                                                                  │
│   71 │   │   │   self.erase()                                                                    │
│   72 │   │                                                                                       │
│   73 │   │   if index_does_not_exist or overwrite != 'reuse':                                    │
│ ❱ 74 │   │   │   self.__launch(collection)                                                       │
│   75 │   │                                                                                       │
│   76 │   │   return self.index_path                                                              │
│   77                                                                                             │
│                                                                                                  │
│ /home/sondors/Documents/price/ColBERT/docs/../colbert/indexer.py:85 in __launch                  │
│                                                                                                  │
│   82 │   │                                                                                       │
│   83 │   │   # Encodes collection into index using the CollectionIndexer class                   │
│   84 │   │   launcher = Launcher(encode)                                                         │
│ ❱ 85 │   │   launcher.launch(self.config, collection, shared_lists, shared_queues)               │
│   86                                                                                             │
│                                                                                                  │
│ /home/sondors/Documents/price/ColBERT/docs/../colbert/infra/launcher.py:76 in launch             │
│                                                                                                  │
│    73 │   │                                                                                      │
│    74 │   │   # TODO: If the processes crash upon join, raise an exception and don't block on    │
│    75 │   │                                                                                      │
│ ❱  76 │   │   return_values = sorted([return_value_queue.get() for _ in all_procs])              │
│    77 │   │   return_values = [val for rank, val in return_values]                               │
│    78 │   │                                                                                      │
│    79 │   │   if not self.return_all:                                                            │
│                                                                                                  │
│ /home/sondors/Documents/price/ColBERT/docs/../colbert/infra/launcher.py:76 in <listcomp>         │
│                                                            

 16%|█▌        | 8/50 [00:48<04:06,  5.88s/it]]

## Search

Having built the index and prepared our `searcher`, we can search for individual query strings.

We can use the `queries` set we loaded earlier — or you can supply your own questions. Feel free to get creative! But keep in mind this set of ~300k lifestyle passages can only answer a small, focused set of questions!

In [ ]:
with Run().context(RunConfig(experiment='notebook')):
    searcher = Searcher(index=index_name)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/sondors/Documents/price/ColBERT/docs/../colbert/infra/config/base_config.py:82 in          │
│ load_from_index                                                                                  │
│                                                                                                  │
│    79 │   │                                                                                      │
│    80 │   │   try:                                                                               │
│    81 │   │   │   metadata_path = os.path.join(index_path, 'metadata.json')                      │
│ ❱  82 │   │   │   loaded_config, _ = cls.from_path(metadata_path)                                │
│    83 │   │   except:                                                                            │
│    84 │   │   │   metadata_path = os.path.join(index_path, 'plan.json')                          │
│    85 │   │   │   loaded_config, _ = cls.from_path(metadata_path)                                │
│                                                                                                  │
│ /home/sondors/Documents/price/ColBERT/docs/../colbert/infra/config/base_config.py:42 in          │
│ from_path                                                                                        │
│                                                                                                  │
│    39 │                                                                                          │
│    40 │   @classmethod                                                                           │
│    41 │   def from_path(cls, name):                                                              │
│ ❱  42 │   │   with open(name) as f:                                                              │
│    43 │   │   │   args = ujson.load(f)                                                           │
│    44 │   │   │                                                                                  │
│    45 │   │   │   if 'config' in args:                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
FileNotFoundError: [Errno 2] No such file or directory: 
'/home/sondors/Documents/price/ColBERT/docs/experiments/notebook/indexes/models.18_categories.2bits/metadata.json'

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 with Run().context(RunConfig(experiment='notebook')):                                        │
│ ❱ 2 │   searcher = Searcher(index=index_name)                                                    │
│   3                                                                                              │
│                                                                                                  │
│ /home/sondors/Documents/price/ColBERT/docs/../colbert/searcher.py:30 in __init__                 │
│                                                                                                  │
│    27 │   │                                                                                      │
│    28 │   │   default_index_root = initial_config.index_root_                                    │
│    29 │   │   self.index = os.path.join(default_index_root, index)                               │
│ ❱  30 │   │   self.index_config = ColBERTConfig.load_from_index(self.index)                      │
│    31 │   │                                                                                      │
│    32 │   │   self.

In [ ]:
offers[37]

'Смартфон Samsung Galaxy Z Flip3 F711B 128Gb Violet'

In [ ]:
query = offers[37]   # or supply your own query

# Find the top-3 passages for this query
results = searcher.search(query, k=5)

# Print out the top-k retrieved passages
for passage_id, passage_rank, passage_score in zip(*results):
    print(f"\t [{passage_rank}] \t\t {passage_score:.1f} \t\t {searcher.collection[passage_id]}")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│   1 query = offers[37]   # or supply your own query                                              │
│   2                                                                                              │
│   3 # Find the top-3 passages for this query                                                     │
│ ❱ 4 results = searcher.search(query, k=5)                                                        │
│   5                                                                                              │
│   6 # Print out the top-k retrieved passages                                                     │
│   7 for passage_id, passage_rank, passage_score in zip(*results):                                │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'searcher' is not defined

## Batch Search

In many applications, you have a large batch of queries and you need to maximize the overall throughput. For that, you can use the `searcher.search_all(queries, k)` method, which returns a `Ranking` object that organizes the results across all queries.

(Batching provides many opportunities for higher-throughput search, though we have not implemented most of those optimizations for compressed indexes yet.)

In [ ]:
import time
start_time = time.time()
rankings = searcher.search_all(offers, k=5).todict()
print(f"time_spent = {time.time() - start_time}")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 import time                                                                                  │
│   2 start_time = time.time()                                                                     │
│ ❱ 3 rankings = searcher.search_all(offers, k=5).todict()                                         │
│   4 print(f"time_spent = {time.time() - start_time}")                                            │
│   5                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'searcher' is not defined

In [ ]:
rankings

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 rankings                                                                                     │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'rankings' is not defined

In [ ]:
searcher.collection[27946]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 searcher.collection[27946]                                                                   │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'searcher' is not defined

In [ ]:
rankings[30]  # For query 30, a list of (passage_id, rank, score) for the top-k passages

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 rankings[30]  # For query 30, a list of (passage_id, rank, score) for the top-k passages     │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'rankings' is not defined

In [ ]:
type(rankings)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 type(rankings)                                                                               │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'rankings' is not defined

In [ ]:
import json

with open(f'{fld_pth}/rankings_1_epoch.json', 'w') as fp:
    json.dump(rankings, fp)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│   1 import json                                                                                  │
│   2                                                                                              │
│   3 with open(f'{fld_pth}/rankings_1_epoch.json', 'w') as fp:                                    │
│ ❱ 4 │   json.dump(rankings, fp)                                                                  │
│   5                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'rankings' is not defined

 94%|█████████▍| 47/50 [02:13<00:08,  2.82s/it]